In [ ]:
# Loading the Packages
%reload_ext autoreload
%autoreload 2

# basic packages
import warnings
warnings.filterwarnings('ignore')
import os
import sys
from pathlib import Path
from tqdm import tqdm
import scanpy as sc

# Apply monkey patch before importing DECIPHER
import matplotlib_inline.backend_inline
import IPython.display
IPython.display.set_matplotlib_formats = matplotlib_inline.backend_inline.set_matplotlib_formats

from decipher import DECIPHER, CFG
from decipher.utils import scanpy_viz

CFG.omics.spatial_graph.mode = 'radius'
CFG.omics.spatial_graph.radius = 300

import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams.update({
    "pgf.texsystem": "xelatex",      # 使用 XeLaTeX，如果不需要 LaTeX 公式渲染，可以省略
    'font.family': 'serif',          # 字体设置为衬线字体
    'text.usetex': False,            # 禁用 LaTeX，使用 Matplotlib 内置文字渲染
    'pgf.rcfonts': False,            # 禁用 pgf 的默认字体管理
    'pdf.fonttype': 42,              # 确保字体为 TrueType 格式，可被 Illustrator 编辑
    'ps.fonttype': 42,               # EPS 文件也使用 TrueType 格式
    'figure.dpi': 300,               # 设置图形分辨率
    'savefig.dpi': 300,              # 保存的图形文件分辨率
    'axes.unicode_minus': False,     # 避免负号问题
})


In [ ]:
# workdir 
BASE_DIR = Path(r'/mnt/g/spatial_data')
RUN_ID_LIST = []

# analysis dir
RUN_ID = '20250222_combined_analysis_of_pseudo_HCC3D'
analysis_dir = BASE_DIR / 'analysis' / RUN_ID
typ_path = analysis_dir / "cell_typing"

output_path = analysis_dir / "DECIPHER"
output_path.mkdir(exist_ok=True)

In [ ]:
# Load the data
combine_adata_st = sc.read_h5ad(typ_path / 'combine_adata_st.h5ad')
adata_direct = sc.read_h5ad(typ_path / 'adata.h5ad')
adata = adata_direct[adata_direct.obs.index.isin(combine_adata_st.obs.index)]
adata.obs = combine_adata_st.obs

# format for later analysis
adata.obs = adata.obs.rename(columns={'X_pos':'X', 'Y_pos':'Y'})
adata.obsm['spatial'] = adata.obs.loc[:, ['X', 'Y', 'Z']].values
adata = adata[adata.obs['type'] != 'other']
adata.X = adata.raw.X.copy()
print(adata)
adata.obs.head()

In [ ]:
# Init model
model = DECIPHER(work_dir=str(output_path / 'model'))

In [ ]:
# Register data (adata.X is raw counts, adata.obsm['spatial'] is spatial coordinates)
model.register_data(adata)

In [ ]:
# Fit DECIPHER model
model.fit_omics()

In [ ]:
# Clustering disentangled embeddings
adata.obsm['X_center'] = model.center_emb  # intracellular molecular embedding
adata.obsm['X_nbr'] =  model.nbr_emb  # spatial context embedding

In [ ]:
adata = scanpy_viz(adata, ['center', 'nbr'], rapids=False, resolution=0.5)

In [ ]:
print(adata)

In [ ]:
# Plot
size=2
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('DECIPHER embeddings')
adata.obsm['X_umap'] = adata.obsm['X_umap_center'].copy()
sc.pl.umap(adata, color=['leiden_center'], ax=ax[0, 0], show=False, s=size)
adata.obsm['X_umap'] = adata.obsm['X_umap_nbr'].copy()
sc.pl.umap(adata, color=['leiden_nbr'], ax=ax[0, 1], show=False, s=size)
sc.pl.embedding(adata, basis="spatial", color=f'leiden_center', s=size, show=False, ax=ax[1, 0])
sc.pl.embedding(adata, basis="spatial", color=f'leiden_nbr', s=size, show=False, ax=ax[1, 1])
plt.tight_layout()
plt.show()